In [57]:
from functools import total_ordering, reduce
import re
import csv
from tqdm import tqdm
from BTrees.OOBTree import OOBTree

In [ ]:
# Lista dei postings aka i docID
class PostingsList:

    def __init__(self) -> None:
        self._postings_list = []

    # crea una PostingsList da una lista di docID e la ordina (forse non necssario)
    @classmethod
    def from_postings_list(cls, postings_list: list[int]) -> 'PostingsList':
        plist = cls()
        postings_list.sort()
        plist._postings_list = postings_list
        return plist

    # Crea una PostingsList da un singolo docID
    @classmethod
    def from_doc_id(cls, doc_id: int):
        plist = cls()
        plist._postings_list = [doc_id]
        return plist

    # Concatena due PostingsList. Le liste sono ordinate e i duplicati rimossi. other contiene una PostingsList creata successivamente a self (i docID saranno piu grandi o uguali)
    def merge(self, other: "PostingsList") -> 'PostingsList':
        i = 0  # Start index for the other PostingList.
        last = self._postings_list[-1]  # The last Posting in the current list.
        # Loop through the other PostingList and skip duplicates.
        while (i < len(other._postings_list) and last == other._postings_list[i]):
            i += 1  # Increment the index if a duplicate is found.
        # Append the non-duplicate postings from the other list.
        self._postings_list += other._postings_list[i:]
        return self

    # Ottiene i titoli di documenti dai docID nella PostingsList
    def get_from_corpus(self, corpus):
        return list(map(lambda x: corpus[x], self._postings_list))

    # Effettua l'intersezione di due PostgingsList con il metodo del doppio indice
    def intersection(self, other: "PostingsList") -> 'PostingsList':
        plist = []
        i = 0  # indice riferito a self
        j = 0  # indice riferito a other
        # finché non si eccede la dimensione di ciascuna lista:
        while (i < len(self._postings_list)) and (j < len(other._postings_list)):
            # se c'e' un match aggiungi l'elemento e incrmeneta entrambi
            if self._postings_list[i] == other._postings_list[j]:
                plist.append(self._postings_list[i])
                i += 1
                j += 1
            # altrimenti aumenta il piu piccolo dei due
            elif self._postings_list[i] <= other._postings_list[j]:
                i += 1
            # altrimenti aumenta l'altro
            else:
                j += 1
        return PostingsList.from_postings_list(plist)

    # Effettua l'unione di due PostingsList con il metodo del doppio indice
    def union(self, other: "PostingsList") -> 'PostingsList':
        plist = []
        i = 0  # indice riferito a self
        j = 0  # indice riferito a other
        # fintanto che gli indici sono piu' piccoli di entrambe le liste
        while (i < len(self._postings_list)) and (j < len(other._postings_list)):
            # aggiungi il docID e aumenta entrambi
            if self._postings_list[i] == other._postings_list[j]:
                plist.append(self._postings_list[i])
                i += 1
                j += 1
            # altrimenti aggiungi il docID e aumenta il piu' piccolo
            elif self._postings_list[i] < other._postings_list[j]:
                plist.append(self._postings_list[i])
                i += 1
            #  aggiungi l'altro e incrementalo
            else:
                plist.append(other._postings_list[j])
                j += 1
        # aggiungi la porzione restante di lista
        if i < len(self._postings_list):  # nel caso in cui self era piu' lunga
            plist += self._postings_list[i:]
        elif j < len(other._postings_list):  # nel caso in cui other era piu' lunga
            plist += other._postings_list[j:]
        return PostingsList.from_postings_list(plist)

    # Effettua la negazione del tipo AND NOT con il metodo dei due indici
    def negation(self, other: 'PostingsList') -> 'PostingsList':
        plist = []
        i = 0
        j = 0
        while (i < len(self._postings_list)) and (j < len(other._postings_list)):
            # se self contiene il docID, scartalo e incrementa entrambi
            if self._postings_list[i] == other._postings_list[j]:
                i += 1
                j += 1
            # aggiungi il docID da self e incrementa se e' piu' piccolo
            elif self._postings_list[i] < other._postings_list[j]:
                plist.append(self._postings_list[i])
                i += 1
            # incrementa other
            else:
                j += 1
        # aggiungi i documenti mancanti da self
        if i < len(self._postings_list):  # se e' piu' lungo di other
            plist += self._postings_list[i:]
        return PostingsList.from_postings_list(plist)

    def __repr__(self) -> str:
        return ", ".join(map(str, self._postings_list))

In [59]:
plist = PostingsList.from_postings_list([1, 2])
plist.negation(PostingsList.from_postings_list([1, 3]))

2

In [ ]:
class ImpossibleMergeException(Exception):
    pass

# Da eliminare imo


@total_ordering
class Term:
    def __init__(self, term: str, doc_id: int) -> None:
        self.term = term
        self.postings_list = PostingsList.from_doc_id(doc_id)

    def merge(self, other: "Term") -> 'Term':
        if self == other:
            self.postings_list.merge(other.postings_list)
        else:
            raise ImpossibleMergeException
        return self

    def __eq__(self, other) -> bool:
        return self.term == other.term

    def __gt__(self, other) -> bool:
        return self.term > other.term

    def __repr__(self) -> str:
        return self.term + ": " + str(self.postings_list)

In [ ]:
def normalize(text):
    # Removes punctuation from the text using a regular expression.
    no_punctuation = re.sub(r'[^\w\s^-]', '', text)
    # Converts the text to lowercase.
    downcase = no_punctuation.lower()
    # Returns the normalized text.
    return downcase


def tokenize(content) -> list:
    normalized = normalize(content)
    return normalized.split()


class InvertedIndex:

    def __init__(self) -> None:
        self.btree = OOBTree()  # usa un Btree per rendere piu' veloci aggiornamenti dell'indice

    @classmethod
    def from_corpus(cls, corpus, max_size=0):
        terms = {}  # dizionario temporaneo per tenere l'indice iniziale
        # per ogni documento
        for doc_id, content in enumerate(tqdm(corpus), max_size):
            # crea un set dei termini che contiene
            tokens = set(tokenize(content.description))
            for token in tokens:  # per ogni termine
                if token in terms:  # se contenuto
                    terms[token].merge(PostingsList.from_doc_id(
                        doc_id))  # fai merge delle PostingsList
                else:  # altrimenti aggiungi
                    terms[token] = PostingsList.from_doc_id(doc_id)
        idx = cls()
        idx.btree.update(terms)
        return idx

    # crea il biword index per le phrase queries
    @classmethod
    def from_corpus_biword(cls, corpus, max_size=0):
        terms = {}
        # per ogni documento
        for doc_id, content in enumerate(tqdm(corpus), max_size):
            tokens = tokenize(content.description)
            # per ogni parola
            for i in range(1, len(tokens)-1):
                token = tokens[i-1]+tokens[i]
                if token in terms:
                    terms[token].merge(PostingsList.from_doc_id(doc_id))
                else:
                    terms[token] = PostingsList.from_doc_id(doc_id)
        idx = cls()
        idx.btree.update(terms)
        return idx

    def __getitem__(self, key: str) -> PostingsList:
        return self.btree[key]

    def __len__(self):
        return len(self.btree)

    def __repr__(self) -> str:
        return self.btree

In [ ]:
class MovieDescription:
    def __init__(self, title: str, description: str):
        self.title = title
        self.description = description

    def __repr__(self) -> str:
        return self.title

# leggi il file descrizione e metadata e crea un corpus (collection di documenti)


def read_movie_description(movie_metadata, description_file):
    names = {}
    corpus = []
    with open(movie_metadata, 'r') as file:  # leggi i metadati
        movie_names = csv.reader(file, delimiter='\t')
        for description in movie_names:  # aggiungi a names la coppia id_film: titolo
            names[description[0]] = description[2]
    with open(description_file, 'r') as file:  # leggi le descrizioni
        descriptions = csv.reader(file, delimiter='\t')
        for description in descriptions:
            try:
                # aggiungi al corpus il titolo e la descrizione di ciascun film
                corpus.append(MovieDescription(
                    # il docID e' la posizione del documento nel corpus
                    names[description[0]], description[1]))
            except KeyError:
                pass
    return corpus

In [ ]:
class IrSystem:
    def __init__(self, corpus: list[MovieDescription], index: InvertedIndex, biword: InvertedIndex, max_size_aux=10000) -> None:
        self._corpus = corpus
        self._index = index  # inverted index
        self._invalid_vec = []  # invalidation bit vector
        self._temp_idx = None  # indice ausiliario
        self.max_size_aux = max_size_aux  # massimo docID assegnato
        self._biword = biword  # inverted index con biword per phrase queries

    # Crea l'indice e il biword
    @classmethod
    def from_corpus(cls, corpus: list[MovieDescription]) -> 'IrSystem':
        index = InvertedIndex.from_corpus(corpus)
        biword = InvertedIndex.from_corpus_biword(corpus)
        ir = cls(corpus, index, biword)
        ir._invalid_vec = [0] * len(corpus)
        return ir

    # Segna documenti cancellati
    def delete_docs(self, documents):
        for doc in documents:
            self._invalid_vec[doc] = 1

    # Aggiungi documenti nuovi all'indice ausiliario
    def add_docs(self, corpus):
        # i nuovi documenti usano docID piu' grandi
        aux = InvertedIndex.from_corpus(corpus, len(self._invalid_vec))
        if self._temp_idx is None:  # se non e' presente nell'indice ausiliario
            self._temp_idx = aux  # aggiungilo
        else:  # altrimenti
            pass  # fai il merge delle PostingsList
        if len(self._temp_idx) > self.max_size_aux:  # se l'indice ausiliario e' troppo grande
            self.merge_idx()  # fai merge
        # aggiorna la dimensione massima attuale
        self.max_size_aux += len(corpus)
        # aggiorna l'invalidation bit vector
        self._invalid_vec += [0] * len(corpus)

    # Merge dell'indice ausilario con l'InvertedIndex
    def merge_idx(self):
        pass

    # Effettua una query booleana combinando i termini con AND, OR e NOT
    def query(self, query: str):
        tokens = query.split()
        # riscrive la query con gli operatori postfix
        postfix = infix_to_postfix(tokens)
        stack = []  # PostingsList ancora da processare
        for token in postfix:
            left = stack.pop()
            right = stack.pop()
            if token == 'AND':  # caso AND, conviene ottimizzare la query facendo l'intersezione delle liste piu' corte in primis
                if not isinstance(left, list):
                    left = [left]
                if not isinstance(right, list):
                    right = [right]
                # aggiungi allo stack una lista [left, right]
                stack.append(left + right)
            elif token in ('OR', 'NOT'):
                if isinstance(left, list):  # se left e' una lista (catena di AND)
                    # effettua la sequenza di AND
                    left = self.optimize_and_query(left)
                if isinstance(right, list):  # se right e' una lista (catena di AND)
                    # effettua la sequenza di AND
                    right = self.optimize_and_query(right)
                if token == 'OR':  # effettua l'OR
                    stack.append(left.union(right))
                else:  # effettua il NOT (AND NOT)
                    # siccome lo stack contiene gli operandi in ordine invertito si invertono left e right
                    stack.append(right.negation(left))
            else:  # aggiungi una PostingsList da processare allo stack
                try:  # prova a cercarla nell'InvertedIndex e in quello ausiliare
                    stack.append(self._index[token].merge(
                        self._temp_idx[token]))
                except KeyError:
                    pass
        result = stack.pop()  # estrai l'ultimo elemento (risultato finale)
        if isinstance(result, list):  # se e' tuttora una lista (= catena di AND), fai l'intersezione
            result = self.optimize_and_query(result)
        for deleted in self._invalid_vec:  # elimina i documenti cancellati
            if deleted:
                result._postings_list.remove(deleted)
        return result.get_from_corpus(self._corpus)

    # Effettua operazioni di AND consecutive facendo l'intersezione di PostingsList piu' corte prima
    def optimize_and_query(self, terms: list[PostingsList]):
        # ordina le PostingsList per lunghezza crescente
        plist = sorted(terms, key=lambda x: len(x._postings_list))
        result = reduce(lambda x, y: x.intersection(y), plist)
        return result

    # Ricerca una sequenza specifica di parola nel corpus con biword
    def phrase_query(self, query: str):
        biword_query = []
        words = query.split()
        for i in range(1, len(words)-1):
            # concatena le parole della query in coppie
            biword_query.append(words[i-1]+words[i])
        # cerca le biword nel biword index
        postings = map(lambda w: self._biword[w], biword_query)
        # effettua l'intersezione delle PostingsList trovate
        plist = reduce(lambda x, y: x.intersection(y), postings)
        return plist.get_from_corpus(self._corpus)

# Rende una espressione da infix a postfix: a AND b OR c -> a b AND c OR


def infix_to_postfix(tokens):
    output = []  # risultato finale
    stack = []  # ancora da processare
    for token in tokens:
        if token in ('AND', 'OR', 'NOT'):  # se e' un operatore
            # finche' ci sono parole da processare e non e' una parentesi
            while (stack and stack[-1] != '('):
                # aggiungi al risultato finale le parole una dopo l'altra
                output.append(stack.pop())
            stack.append(token)  # aggiungi l'operatore allo stack
        elif token == '(':  # aggiungi la parentesi allo stack
            stack.append(token)
        elif token == ')':
            # fino a che non incontro la parentesi aperta o si svuota lo stack
            while stack and stack[-1] != '(':
                # aggiungo all'output il contenuto dello stack
                output.append(stack.pop())
            stack.pop()  # remove '('
        else:  # aggiungi un termine all'outpuit
            output.append(token)
    while stack:  # svuota lo stack
        output.append(stack.pop())
    return output

In [64]:
corpus = read_movie_description(
    '../Code IR/data/movie.metadata.tsv', '../Code IR/data/plot_summaries.txt')

In [65]:
ir = IrSystem.from_corpus(corpus)

100%|██████████| 42204/42204 [00:21<00:00, 1939.76it/s]


In [68]:
ir.phrase_query('speak during meetings')

[The Neighbour No. 13, Lord of the Flies, Andha Naal, The Lover, Star Runners]

In [ ]:
print(len(ir.query('yoda')), len(ir.query(
    'luke')), len(ir.query('wars')))

13 161 179


In [ ]:
ir.query('luke')

[Afghan Luke,
 Daisy Town,
 Decoys 2: Alien Seduction,
 Out Cold,
 2:37,
 Lilies of the Field,
 Scumbus,
 Death of a Gunfighter,
 Fatty and Mabel Adrift,
 Santa Baby,
 The Boys Club,
 SpaceCamp,
 Undiscovered,
 Fast Five,
 Star Wars Episode V: The Empire Strikes Back,
 Dual,
 Angels and Demons,
 Children of Men,
 Spiderhole,
 Spike and Suzy: The Texas Rangers,
 Children of the Corn V: Fields of Terror,
 Stagecoach,
 Animal Kingdom,
 The Prince of Tides,
 The Dukes of Hazzard: Reunion!,
 Vanishing on 7th Street,
 Green Light,
 Still Crazy,
 Coming Home,
 Decoys,
 Halloween Resurrection,
 Imaginationland Episode II,
 Slaves,
 Jennifer,
 Nagarangalil Chennu Raparkam,
 Star Wars Episode IV: A New Hope,
 Memphis Belle,
 Wishology,
 The Wendell Baker Story,
 The Little Troll Prince: A Christmas Parable,
 Mustang Country,
 Macon County Line,
 The Long Kiss Goodnight,
 The Dukes of Hazzard: Hazzard in Hollywood!,
 A Woman's Secret,
 No Name on the Bullet,
 Tanner on Tanner,
 The Toy that Saved

In [ ]:
ir.query('wars')

[All Quiet on the Western Front,
 American Drug War: The Last White Hope,
 O Stratis parastratise,
 Dobrynya Nikitich and Zmey Gorynych,
 Kind Hearts and Coronets,
 Hardware Wars,
 St. Ives,
 Fido,
 Invitation to the Waltz,
 Alexander the Great,
 Die Abenteuer des Werner Holt,
 La Bandera,
 Paan Singh Tomar,
 The Emperor's Shadow,
 Home of the Brave,
 SpaceCamp,
 Mask,
 Life at the End of the Rainbow,
 Captain Horatio Hornblower,
 Quo Vadis,
 From Star Wars to Jedi: The Making of a Saga,
 G.O.R.A.,
 Grasshoppers,
 Mr. and Mrs. Iyer,
 Zombie Strippers,
 Cat City,
 If I Should Fall,
 St. George Shoots the Dragon,
 Iluminados Por El Fuego,
 Time Bandits,
 Padme,
 The Saragossa Manuscript,
 First Knight,
 The Parade,
 Gone with the Wind,
 The Trojan Women,
 Blue Gold: World Water Wars,
 Star Wars Episode II: Attack of the Clones,
 La famiglia,
 HMS Defiant,
 Wishology,
 An Inconvenient Tax,
 Love and Death,
 Star Odyssey,
 Krrish,
 X-Men Origins: Wolverine,
 Zack and Miri Make a Porno,
 Th